In [1]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

/home/lvanel/anaconda3/envs/lo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import numpy as np

model_name = "astrosbd/french_emotion_camembert" 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForSequenceClassification.from_pretrained(model_name)
config = model.config

config.id2label

{0: 'sad', 1: 'fear', 2: 'anger', 3: 'neutral', 4: 'surprise', 5: 'joy'}

In [3]:
dataset = load_dataset("csv", data_files={"train":"emotions.csv"})
cols = ['tristesse', 'peur', 'colère', 'neutre', 'surprise', 'joie']

print(len(cols))

def one_hot_encode(ds):
    dialogs = []
    ohe_dic= {x:[] for x in cols}
    for i in tqdm(range(len(ds))):
        text = ds['text'][i]
        dialogs.append(text)
        labels = ds['emotion'][i]

        if i == 0:
            print(labels)
            print(ohe_dic.keys())

        for k in ohe_dic.keys():
            if k in labels:
                ohe_dic[k].append(1)
            
            else:
                ohe_dic[k].append(0)

    
    ohe_dic['input'] = dialogs

    return ohe_dic


ohe_dic = one_hot_encode(dataset['train'])
df = pd.DataFrame(ohe_dic)


Generating train split: 161 examples [00:00, 23551.18 examples/s]


6


100%|██████████| 161/161 [00:00<00:00, 3241.67it/s]

joie
dict_keys(['tristesse', 'peur', 'colère', 'neutre', 'surprise', 'joie'])


In [4]:
test = df.iloc[:40]
train = df.iloc[40:]

In [5]:
test.to_csv('emotions_ohe_test.csv', index=False, encoding='UTF-8')
train.to_csv('emotions_ohe_train.csv', index=False, encoding='UTF-8')